In [14]:
import requests
import re
import time
from bs4 import BeautifulSoup

cardrush_url = "https://www.cardrush-pokemon.jp/phone/new"

#TODO: Change this when backend is published
db_url = "http://127.0.0.1:5000"
url = f"{cardrush_url}/new?page=1"


#
def extract_goods_and_prices(soup):
    results = {}

    item_data_divs = soup.find_all('div', class_='item_data')

    for div in item_data_divs:
        goods_name = div.find('span', class_='goods_name')
        selling_price = div.find('p', class_='selling_price')
        model_number = div.find('span', class_ ='model_number')

        if goods_name and selling_price:
            #Extracting name of the card
            goods_name_text = goods_name.text.strip()
            #Extracting ID of the card
            match = re.search(r'\{(\d+/\d+)\}', goods_name_text)
            if match:
                goods_id = match.group(1)
                encoded_goods_id = goods_id.replace('/', '>')
                
                #Extracting the price of the card
                selling_price_text = selling_price.text.strip()
                price = int(re.sub(r'\D+', '', selling_price_text))
                
                #Extracting the model of the card
                model_number_text = model_number.text.strip()
                model_value = re.search(r'S([^]]+)\]', model_number_text)

                if model_value:
                    model_number = 'S' + model_value.group(1)
                    result = {
                        'unique_id': model_number + '-' + encoded_goods_id,
                        'JP_name': goods_name_text,
                        'JP_price': price,
                        'US_price': -1,
                        'US_name': ''
                    }
                    results[result['unique_id']] = result

    return results

def fetch_and_parse(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        print(f"Failed to fetch the page, status code: {response.status_code}")
        return None

#Updating the cards everything certain time
#TODO: Update me when deploying the app

time_interval = 1 * 60 #Time interval is now set to 1 minutes

while True:
    print("Updating Japanese cards in the database")
    soup = fetch_and_parse(url)
    
    if soup:
        # Extract goods_name and selling_price from the first page
        all_results = extract_goods_and_prices(soup)
        # Find the last page number
        pagebottom_div = soup.find('div', id='pagerbottom')

        if pagebottom_div:
            last_page_link = pagebottom_div.find_all('a', class_='pager_btn')[-2]
            last_page_number = int(last_page_link['href'].split('=')[-1])
            
            
            # Loop through the remaining pages
            for page_number in range(2, last_page_number + 1):
                print("Reading page " + str(page_number))
                page_url = f"{cardrush_url}/new?page={page_number}"
                soup = fetch_and_parse(page_url)

                if soup:
                    # Extract goods_name and selling_price from the current page
                    results = extract_goods_and_prices(soup)
                    all_results.update(results)
                
    print(len(all_results))
    # Print the extracted data
    for result in all_results.values():
        get_response = requests.get(db_url + '/get_data/' + result['unique_id'])
        
        if "output" in get_response.json().keys():
            current_card = get_response.json().get("output")
            different_info = {}
            if current_card['JP_name'] != result['JP_name']:
                different_info["JP_name"] = result['JP_name']
            if current_card['JP_price'] != result['JP_price']:
                different_info["JP_price"] = result['JP_price']    
            
            if different_info:
                update_response = requests.post(db_url+ "/update_data/" + result['unique_id'], json=different_info)
                print(update_response.json())
                print("\n")
            else:
                print("id: " + result['unique_id'] + ", " + result['JP_name'] + " is up-to-date!")
        else:
            print("Found a new card! Storing in database now")
            data = {
                "unique_id": result['unique_id'],
                "JP_name": result['JP_name'],
                "JP_price": result['JP_price'],
                "US_name": result['US_name'],
                "US_price": result['US_price']
            }
            store_response = requests.post(db_url+ "/store_data", json=data)
            print(store_response.json())
            print("\n")
    

    print("Finished updating Japanese Pokemon Card Info")

    time.sleep(time_interval)




Updating Japanese cards in the database
Reading page 2
Reading page 3
Reading page 4
Reading page 5
Reading page 6
Reading page 7
Reading page 8
Reading page 9
Reading page 10
Reading page 11
Reading page 12
Reading page 13
Reading page 14
Reading page 15
Reading page 16
Reading page 17
Reading page 18
Reading page 19
Reading page 20
Reading page 21
Reading page 22
Reading page 23
Reading page 24
Reading page 25
Reading page 26
Reading page 27
Reading page 28
Reading page 29
Reading page 30
Reading page 31
Reading page 32
Reading page 33
Reading page 34
1452
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': '〔状態B〕ラティアス【AR】{195/172}', 'JP_price': 220, 'US_name': '', 'US_price': -1, '_id': 'S12a-195>172', 'unique_id': 'S12a-195>172'}}
{}
id: S12a-195>172, 〔状態B〕ラティアス【AR】{195/172} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'かがやくリザードン【K】{015/172}', 'JP_price': 680, 'US_name': '', 'US_price': -1, '_id': 'S12a-015>172',

Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'チオンジェンex【SR】{084/071}', 'JP_price': 780, 'US_name': '', 'US_price': -1, '_id': 'SV2P-084>071', 'unique_id': 'SV2P-084>071'}}
{}
id: SV2P-084>071, チオンジェンex【SR】{084/071} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'パオジアンex【SR】{085/071}', 'JP_price': 1480, 'US_name': '', 'US_price': -1, '_id': 'SV2P-085>071', 'unique_id': 'SV2P-085>071'}}
{}
id: SV2P-085>071, パオジアンex【SR】{085/071} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'ヤドキングex【SR】{086/071}', 'JP_price': 1180, 'US_name': '', 'US_price': -1, '_id': 'SV2P-086>071', 'unique_id': 'SV2P-086>071'}}
{}
id: SV2P-086>071, ヤドキングex【SR】{086/071} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'コノヨザルex【SR】{087/071}', 'JP_price': 780, 'US_name': '', 'US_price': -1, '_id': 'SV2P-087>071', 'unique_id': 'SV2P-087>071'}}
{}
id: SV2P-087>071, 

Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'コオリッポ【AR】{074/071}', 'JP_price': 220, 'US_name': '', 'US_price': -1, '_id': 'SV2P-074>071', 'unique_id': 'SV2P-074>071'}}
{}
id: SV2P-074>071, コオリッポ【AR】{074/071} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'セビエ【AR】{075/071}', 'JP_price': 480, 'US_name': '', 'US_price': -1, '_id': 'SV2P-075>071', 'unique_id': 'SV2P-075>071'}}
{}
id: SV2P-075>071, セビエ【AR】{075/071} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'セゴール【AR】{076/071}', 'JP_price': 220, 'US_name': '', 'US_price': -1, '_id': 'SV2P-076>071', 'unique_id': 'SV2P-076>071'}}
{}
id: SV2P-076>071, セゴール【AR】{076/071} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'セグレイブ【AR】{077/071}', 'JP_price': 680, 'US_name': '', 'US_price': -1, '_id': 'SV2P-077>071', 'unique_id': 'SV2P-077>071'}}
{}
id: SV2P-077>071, セグレイブ【AR】{077/071} is up

Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'デデンネex【SR】{089/073}', 'JP_price': 1980, 'US_name': '', 'US_price': -1, '_id': 'SV1a-089>073', 'unique_id': 'SV1a-089>073'}}
{}
id: SV1a-089>073, デデンネex【SR】{089/073} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'ルガルガンex【SR】{090/073}', 'JP_price': 780, 'US_name': '', 'US_price': -1, '_id': 'SV1a-090>073', 'unique_id': 'SV1a-090>073'}}
{}
id: SV1a-090>073, ルガルガンex【SR】{090/073} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'パルデアドオーex【SR】{091/073}', 'JP_price': 780, 'US_name': '', 'US_price': -1, '_id': 'SV1a-091>073', 'unique_id': 'SV1a-091>073'}}
{}
id: SV1a-091>073, パルデアドオーex【SR】{091/073} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'キハダ【SR】{092/073}', 'JP_price': 11800, 'US_name': '', 'US_price': -1, '_id': 'SV1a-092>073', 'unique_id': 'SV1a-092>073'}}
{}
id: SV1a-092>073, キハダ

Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'ブーピッグ【U】{037/073}', 'JP_price': 30, 'US_name': '', 'US_price': -1, '_id': 'SV1a-037>073', 'unique_id': 'SV1a-037>073'}}
{}
id: SV1a-037>073, ブーピッグ【U】{037/073} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'デデンネex【RR】{038/073}', 'JP_price': 80, 'US_name': '', 'US_price': -1, '_id': 'SV1a-038>073', 'unique_id': 'SV1a-038>073'}}
{}
id: SV1a-038>073, デデンネex【RR】{038/073} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'ヤレユータン【U】{039/073}', 'JP_price': 30, 'US_name': '', 'US_price': -1, '_id': 'SV1a-039>073', 'unique_id': 'SV1a-039>073'}}
{}
id: SV1a-039>073, ヤレユータン【U】{039/073} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'カヌチャン【C】{040/073}', 'JP_price': 30, 'US_name': '', 'US_price': -1, '_id': 'SV1a-040>073', 'unique_id': 'SV1a-040>073'}}
{}
id: SV1a-040>073, カヌチャン【C】{040/073} is up

...

Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'ドーミラー【-】{009/030}', 'JP_price': 50, 'US_name': '', 'US_price': -1, '_id': 'SK-009>030', 'unique_id': 'SK-009>030'}}
{}
id: SK-009>030, ドーミラー【-】{009/030} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'チェリム【-】{002/030}', 'JP_price': 50, 'US_name': '', 'US_price': -1, '_id': 'SK-002>030', 'unique_id': 'SK-002>030'}}
{}
id: SK-002>030, チェリム【-】{002/030} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'チェリンボ【-】{001/030}', 'JP_price': 50, 'US_name': '', 'US_price': -1, '_id': 'SK-001>030', 'unique_id': 'SK-001>030'}}
{}
id: SK-001>030, チェリンボ【-】{001/030} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': 'オリジンパルキアVSTAR【RRR】{023/067}', 'JP_price': 120, 'US_name': '', 'US_price': -1, '_id': 'S10P-023>067', 'unique_id': 'S10P-023>067'}}
{}
id: S10P-023>067, オリジンパルキアVSTAR【RRR】{023/067} is up-to-d

Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': '〔状態B〕レスキューキャリー【-】{142/172}', 'JP_price': 50, 'US_name': '', 'US_price': -1, '_id': 'S12a-142>172', 'unique_id': 'S12a-142>172'}}
{}
id: S12a-142>172, 〔状態B〕レスキューキャリー【-】{142/172} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': '〔状態B〕ミラージュゲート【-】{141/172}', 'JP_price': 50, 'US_name': '', 'US_price': -1, '_id': 'S12a-141>172', 'unique_id': 'S12a-141>172'}}
{}
id: S12a-141>172, 〔状態B〕ミラージュゲート【-】{141/172} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': '〔状態B〕ヒスイのヘビーボール【-】{140/172}', 'JP_price': 50, 'US_name': '', 'US_price': -1, '_id': 'S12a-140>172', 'unique_id': 'S12a-140>172'}}
{}
id: S12a-140>172, 〔状態B〕ヒスイのヘビーボール【-】{140/172} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': '〔状態B〕パワータブレット【-】{139/172}', 'JP_price': 50, 'US_name': '', 'US_price': -1, '_id': 'S12a-139>172', 'unique_id': 'S12a

Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': '〔状態A-〕バケッチャ(ミラー)【-】{061/172}', 'JP_price': 110, 'US_name': '', 'US_price': -1, '_id': 'S12a-061>172', 'unique_id': 'S12a-061>172'}}
{}
id: S12a-061>172, 〔状態A-〕バケッチャ(ミラー)【-】{061/172} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': '〔状態A-〕サマヨール(ミラー)【-】{058/172}', 'JP_price': 40, 'US_name': '', 'US_price': -1, '_id': 'S12a-058>172', 'unique_id': 'S12a-058>172'}}
{}
id: S12a-058>172, 〔状態A-〕サマヨール(ミラー)【-】{058/172} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': '〔状態A-〕ヨマワル(ミラー)【-】{057/172}', 'JP_price': 40, 'US_name': '', 'US_price': -1, '_id': 'S12a-057>172', 'unique_id': 'S12a-057>172'}}
{}
id: S12a-057>172, 〔状態A-〕ヨマワル(ミラー)【-】{057/172} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': '〔状態A-〕ルナトーン(ミラー)【-】{056/172}', 'JP_price': 70, 'US_name': '', 'US_price': -1, '_id': 'S12a-056>172', 'uni

Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': '〔状態B〕ミュウV【RR】{053/172}', 'JP_price': 50, 'US_name': '', 'US_price': -1, '_id': 'S12a-053>172', 'unique_id': 'S12a-053>172'}}
{}
id: S12a-053>172, 〔状態B〕ミュウV【RR】{053/172} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': '〔状態B〕スイクンV【RR】{024/172}', 'JP_price': 50, 'US_name': '', 'US_price': -1, '_id': 'S12a-024>172', 'unique_id': 'S12a-024>172'}}
{}
id: S12a-024>172, 〔状態B〕スイクンV【RR】{024/172} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': '〔状態B〕リザードンV【RR】{013/172}', 'JP_price': 80, 'US_name': '', 'US_price': -1, '_id': 'S12a-013>172', 'unique_id': 'S12a-013>172'}}
{}
id: S12a-013>172, 〔状態B〕リザードンV【RR】{013/172} is up-to-date!
Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': '〔状態A-〕ラルトス(ミラー)【C】{032/067}', 'JP_price': 630, 'US_name': '', 'US_price': -1, '_id': 'S9a-032>067', 'unique_id': 'S9a-032>067'}}
{}
id

KeyboardInterrupt: 

In [15]:
import datetime
print("Next update is in " + str(datetime.timedelta(seconds=3600)))

Next update is in 1:00:00


In [13]:
get_response = requests.get(db_url + '/get_data/' + "S12a-195>172")
print("Response:")
print(get_response.json())
print(get_response.json().keys())

if "output" in get_response.json().keys():
    print("yes")
    current_card = get_response.json().get("output")
    different_info = {}
    if current_card['JP_name'] != result['JP_name']:
        different_info["JP_name"] = result['JP_name']
    if current_card['JP_price'] != result['JP_price']:
        different_info["JP_price"] = result['JP_price']    

    print(different_info)
    
    if different_info:
        update_response = requests.post(db_url+ "/update_data/" + result['unique_id'], json=different_info)
        print(update_response.json())
        print("\n")
    else:
        print("id: " + result['unique_id'] + ", " + result['JP_name'] + " is up-to-date!")

Response:
{'message': 'Data retrieved successfully!', 'output': {'JP_name': '〔状態B〕ラティアス【AR】{195/172}', 'JP_price': 220, 'US_name': '', 'US_price': -1, '_id': 'S12a-195>172', 'unique_id': 'S12a-195>172'}}
dict_keys(['message', 'output'])
yes
{}
id: S12a-195>172, 〔状態B〕ラティアス【AR】{195/172} is up-to-date!
